In [1]:
import os
import os.path as osp
import sys
sys.path.append('/home/damon/Documents/')
import numpy as np
import torch, torchvision
import torch.nn.functional as F
from torch import nn
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from torchsummary import summary

from torchviz import make_dot, make_dot_from_trace
import torch
import torchvision
import torchvision.transforms as transforms

import mmdet
import mmcv
import deformable as D

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(root='/tmp/data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='/tmp/data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [3]:
import torch.optim as optim
lr = 1e-3
criterion = nn.CrossEntropyLoss()

# Training
def train(epoch, net):
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if batch_idx % 128 == 0:
            print('train Loss: %.3f | Acc: %.3f%% (%d/%d)'
                % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))

def test(epoch, net):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            if batch_idx  % 128 == 0:
                print('test Loss: %.3f | Acc: %.3f%% (%d/%d)'
                    % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

In [30]:
conv_op = nn.Conv2d
conv_op = D.DeformConv2d
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = conv_op(3, 32, 3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = conv_op(32, 64, 3, stride=2, padding=1)
        self.conv3 = conv_op(64, 128, 3, stride=2, padding=1)
        self.conv4 = conv_op(128, 256, 3, stride=2, padding=1)
        self.conv1x1 = nn.Conv2d(256, 512, 1, stride=1, padding=0)
        self.fc = nn.Linear(256, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = x.view(-1, 256)
        x = self.fc(x)
        return x

net = Net().cuda()

In [31]:
from time import time
a = time()
for epoch in range(start_epoch, start_epoch+50):
    train(epoch, net)
    test(epoch, net)
print(time() - a)


Epoch: 0
train Loss: 2.301 | Acc: 9.375% (12/128)
train Loss: 2.298 | Acc: 13.608% (2247/16512)
train Loss: 2.294 | Acc: 14.449% (4753/32896)
train Loss: 2.287 | Acc: 15.158% (7470/49280)
test Loss: 2.237 | Acc: 21.000% (21/100)

Epoch: 1
train Loss: 2.256 | Acc: 18.750% (24/128)
train Loss: 2.236 | Acc: 19.265% (3181/16512)
train Loss: 2.205 | Acc: 20.088% (6608/32896)
train Loss: 2.176 | Acc: 20.980% (10339/49280)
test Loss: 2.008 | Acc: 35.000% (35/100)

Epoch: 2
train Loss: 2.104 | Acc: 25.781% (33/128)
train Loss: 2.076 | Acc: 24.673% (4074/16512)
train Loss: 2.045 | Acc: 26.246% (8634/32896)
train Loss: 2.011 | Acc: 27.524% (13564/49280)
test Loss: 1.800 | Acc: 40.000% (40/100)

Epoch: 3
train Loss: 1.968 | Acc: 25.781% (33/128)
train Loss: 1.889 | Acc: 31.256% (5161/16512)
train Loss: 1.859 | Acc: 32.116% (10565/32896)
train Loss: 1.840 | Acc: 32.946% (16236/49280)
test Loss: 1.736 | Acc: 44.000% (44/100)

Epoch: 4
train Loss: 1.735 | Acc: 39.062% (50/128)
train Loss: 1.751 | A

KeyboardInterrupt: 

In [6]:
from time import time 
conv_op = D.DeformConv2d
class DNet(nn.Module):
    def __init__(self):
        super(DNet, self).__init__()
        self.conv1 = conv_op(3, 6, 3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = conv_op(6, 16, 3, stride=1, padding=1)
        self.fc1 = nn.Linear(16 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

dnet = DNet().cuda()
a = time()
for epoch in range(start_epoch, start_epoch+200):
    train(epoch, dnet)
    test(epoch, dnet)
print(time() - a)


Epoch: 0
train Loss: 2.304 | Acc: 8.594% (11/128)
train Loss: 2.297 | Acc: 10.338% (1707/16512)
train Loss: 2.289 | Acc: 12.415% (4084/32896)
train Loss: 2.275 | Acc: 12.977% (6395/49280)
test Loss: 2.187 | Acc: 20.000% (20/100)

Epoch: 1
train Loss: 2.193 | Acc: 17.969% (23/128)
train Loss: 2.203 | Acc: 18.193% (3004/16512)
train Loss: 2.176 | Acc: 19.972% (6570/32896)
train Loss: 2.150 | Acc: 21.571% (10630/49280)
test Loss: 2.025 | Acc: 34.000% (34/100)

Epoch: 2
train Loss: 2.070 | Acc: 25.781% (33/128)
train Loss: 2.048 | Acc: 25.987% (4291/16512)
train Loss: 2.027 | Acc: 26.553% (8735/32896)
train Loss: 2.006 | Acc: 27.108% (13359/49280)
test Loss: 1.894 | Acc: 38.000% (38/100)

Epoch: 3
train Loss: 1.961 | Acc: 29.688% (38/128)
train Loss: 1.930 | Acc: 29.784% (4918/16512)
train Loss: 1.910 | Acc: 30.356% (9986/32896)
train Loss: 1.893 | Acc: 30.773% (15165/49280)
test Loss: 1.776 | Acc: 41.000% (41/100)

Epoch: 4
train Loss: 1.770 | Acc: 36.719% (47/128)
train Loss: 1.817 | Ac

train Loss: 1.140 | Acc: 59.466% (9819/16512)
train Loss: 1.130 | Acc: 59.560% (19593/32896)
train Loss: 1.122 | Acc: 59.923% (29530/49280)
test Loss: 0.978 | Acc: 68.000% (68/100)

Epoch: 36
train Loss: 1.023 | Acc: 63.281% (81/128)
train Loss: 1.107 | Acc: 60.556% (9999/16512)
train Loss: 1.115 | Acc: 60.168% (19793/32896)
train Loss: 1.116 | Acc: 60.213% (29673/49280)
test Loss: 0.936 | Acc: 70.000% (70/100)

Epoch: 37
train Loss: 1.123 | Acc: 61.719% (79/128)
train Loss: 1.106 | Acc: 60.816% (10042/16512)
train Loss: 1.113 | Acc: 60.506% (19904/32896)
train Loss: 1.109 | Acc: 60.623% (29875/49280)
test Loss: 0.980 | Acc: 73.000% (73/100)

Epoch: 38
train Loss: 1.027 | Acc: 60.938% (78/128)
train Loss: 1.101 | Acc: 60.538% (9996/16512)
train Loss: 1.103 | Acc: 60.597% (19934/32896)
train Loss: 1.099 | Acc: 60.840% (29982/49280)
test Loss: 0.973 | Acc: 64.000% (64/100)

Epoch: 39
train Loss: 1.149 | Acc: 58.594% (75/128)
train Loss: 1.090 | Acc: 61.252% (10114/16512)
train Loss: 1.09

train Loss: 0.915 | Acc: 67.617% (11165/16512)
train Loss: 0.909 | Acc: 67.799% (22303/32896)
train Loss: 0.915 | Acc: 67.581% (33304/49280)
test Loss: 0.844 | Acc: 72.000% (72/100)

Epoch: 71
train Loss: 0.798 | Acc: 77.344% (99/128)
train Loss: 0.909 | Acc: 67.745% (11186/16512)
train Loss: 0.913 | Acc: 67.707% (22273/32896)
train Loss: 0.910 | Acc: 67.877% (33450/49280)
test Loss: 0.813 | Acc: 68.000% (68/100)

Epoch: 72
train Loss: 1.005 | Acc: 59.375% (76/128)
train Loss: 0.903 | Acc: 68.496% (11310/16512)
train Loss: 0.908 | Acc: 67.993% (22367/32896)
train Loss: 0.908 | Acc: 67.999% (33510/49280)
test Loss: 0.813 | Acc: 74.000% (74/100)

Epoch: 73
train Loss: 0.829 | Acc: 68.750% (88/128)
train Loss: 0.899 | Acc: 68.411% (11296/16512)
train Loss: 0.900 | Acc: 68.577% (22559/32896)
train Loss: 0.902 | Acc: 68.488% (33751/49280)
test Loss: 0.851 | Acc: 70.000% (70/100)

Epoch: 74
train Loss: 0.863 | Acc: 71.875% (92/128)
train Loss: 0.903 | Acc: 67.981% (11225/16512)
train Loss: 0

train Loss: 0.819 | Acc: 71.076% (11736/16512)
train Loss: 0.827 | Acc: 70.945% (23338/32896)
train Loss: 0.825 | Acc: 70.938% (34958/49280)
test Loss: 0.926 | Acc: 68.000% (68/100)

Epoch: 106
train Loss: 0.672 | Acc: 75.781% (97/128)
train Loss: 0.829 | Acc: 70.985% (11721/16512)
train Loss: 0.823 | Acc: 71.255% (23440/32896)
train Loss: 0.822 | Acc: 71.132% (35054/49280)
test Loss: 0.842 | Acc: 71.000% (71/100)

Epoch: 107
train Loss: 0.828 | Acc: 73.438% (94/128)
train Loss: 0.805 | Acc: 71.730% (11844/16512)
train Loss: 0.812 | Acc: 71.483% (23515/32896)
train Loss: 0.818 | Acc: 71.264% (35119/49280)
test Loss: 0.848 | Acc: 73.000% (73/100)

Epoch: 108
train Loss: 0.908 | Acc: 71.094% (91/128)
train Loss: 0.809 | Acc: 71.500% (11806/16512)
train Loss: 0.814 | Acc: 71.337% (23467/32896)
train Loss: 0.815 | Acc: 71.313% (35143/49280)
test Loss: 0.858 | Acc: 73.000% (73/100)

Epoch: 109
train Loss: 0.906 | Acc: 66.406% (85/128)
train Loss: 0.810 | Acc: 71.512% (11808/16512)
train Los


Epoch: 140
train Loss: 0.972 | Acc: 68.750% (88/128)
train Loss: 0.772 | Acc: 73.141% (12077/16512)
train Loss: 0.778 | Acc: 72.781% (23942/32896)
train Loss: 0.775 | Acc: 72.800% (35876/49280)
test Loss: 0.816 | Acc: 73.000% (73/100)

Epoch: 141
train Loss: 0.956 | Acc: 70.312% (90/128)
train Loss: 0.759 | Acc: 73.468% (12131/16512)
train Loss: 0.764 | Acc: 73.085% (24042/32896)
train Loss: 0.767 | Acc: 73.011% (35980/49280)
test Loss: 0.820 | Acc: 73.000% (73/100)

Epoch: 142
train Loss: 0.799 | Acc: 67.188% (86/128)
train Loss: 0.775 | Acc: 72.481% (11968/16512)
train Loss: 0.774 | Acc: 72.723% (23923/32896)
train Loss: 0.772 | Acc: 72.794% (35873/49280)
test Loss: 0.798 | Acc: 71.000% (71/100)

Epoch: 143
train Loss: 0.644 | Acc: 78.906% (101/128)
train Loss: 0.757 | Acc: 73.286% (12101/16512)
train Loss: 0.774 | Acc: 72.881% (23975/32896)
train Loss: 0.767 | Acc: 73.072% (36010/49280)
test Loss: 0.780 | Acc: 75.000% (75/100)

Epoch: 144
train Loss: 0.714 | Acc: 76.562% (98/128)
t

test Loss: 0.754 | Acc: 73.000% (73/100)

Epoch: 175
train Loss: 0.661 | Acc: 79.688% (102/128)
train Loss: 0.741 | Acc: 73.855% (12195/16512)
train Loss: 0.736 | Acc: 74.182% (24403/32896)
train Loss: 0.738 | Acc: 74.131% (36532/49280)
test Loss: 0.785 | Acc: 74.000% (74/100)

Epoch: 176
train Loss: 0.637 | Acc: 82.031% (105/128)
train Loss: 0.727 | Acc: 74.715% (12337/16512)
train Loss: 0.733 | Acc: 74.273% (24433/32896)
train Loss: 0.734 | Acc: 74.203% (36567/49280)
test Loss: 0.773 | Acc: 72.000% (72/100)

Epoch: 177
train Loss: 0.621 | Acc: 76.562% (98/128)
train Loss: 0.729 | Acc: 74.237% (12258/16512)
train Loss: 0.729 | Acc: 74.273% (24433/32896)
train Loss: 0.733 | Acc: 74.156% (36544/49280)
test Loss: 0.797 | Acc: 74.000% (74/100)

Epoch: 178
train Loss: 0.729 | Acc: 73.438% (94/128)
train Loss: 0.731 | Acc: 74.358% (12278/16512)
train Loss: 0.730 | Acc: 74.191% (24406/32896)
train Loss: 0.731 | Acc: 74.249% (36590/49280)
test Loss: 0.766 | Acc: 74.000% (74/100)

Epoch: 179
t